# Exploratory Analysis of Spatial Data: Spatial Autocorrelation #


In this lab we will practise to analyze global and local spatial autocorrelation to analyze spatial data.

The analyses of spatial autocorrelation are also called Exploratory Spatial Data Analysis (ESDA).

This lab is modified from the [tutorial of pysal ESDA](https://pysal.org/).

#### Before the lab, please install the python module esda and splot.

Open anaconda prompt in your start menu as an administrator user. 

Remember to open the right version (Anaconda 3.6) if you have multiple anaconda installed.

Install the module **esda** and **splot** by typing the following command
    - pip install esda
    - pip install splot

#### Import python modules needed for this lab

In [ ]:
import esda
import geopandas as gpd
import libpysal
from libpysal import examples
import numpy as np
import matplotlib.pyplot as plt
import mapclassify as mc
import pysal

from splot.libpysal import plot_spatial_weights
from libpysal.weights.contiguity import Queen, Rook

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

## Download Data

Please download the data from [here](https://drive.google.com/file/d/1k_KF_C7muXAdcRH1FSPhVpBH8mv-ANfs/view?usp=sharing). and extract it to a local folder in your computer.

Please change the file path accordingly.

## Spatial Weight Matrix

Load the dataset `nbr.shp` into a `geopandas` geodataframe.

In [ ]:
gdf = gpd.read_file('C:/Users/yi/Documents/UH_work/Teaching/GEOG389/labs/lab5_data/texas.shp')

Print the first 5 rows of the data frame

Plot the shapefile

In [ ]:
gdf.plot()

As you learned in class, neighborhoods (nearness) can be defined by contiguity, distance threshold (binary) or distance decay function (continuous).

The following codes generate spatial weight matrix using queen contiguity. 

![](images/fig14.jpg)

In [ ]:
wq = Queen.from_dataframe(gdf)

Print the weights of each polygon

In [ ]:
wq.weights

print the neighbors of each polygon

In [ ]:
wq.neighbors

Standardize the weight matrix by rows

In [ ]:
wq.transform='r'

In [ ]:
wq.weights

#### Plotting the neighborhood relations as a network
The map below show the neighborhood relations between the polygons (ciby boundaries) by lines linking their centroids. Polygons that are neighbors to each other are linked.

In [ ]:
plot_spatial_weights(wq, gdf)
plt.show()

---

# Question 1

Please create a plot showing the neighborhood network based on the rook contiguity.

In [ ]:
wr = Rook.from_dataframe(gdf)
plot_spatial_weights(wr, gdf)
plt.show()

---

## Preview the Berlin airbnb dataset

Load the polygon shapefile of Berline neighborhoods. 

In [ ]:
gdf = gpd.read_file('C:/Users/yi/Documents/UH_work/Teaching/GEOG389/labs/lab5_data/nbr.shp')

The data set comes from the Berlin airbnb scrape taken in April 2018. This dataframe was constructed as part of the [GeoPython 2018 workshop](https://github.com/ljwolf/geopython) by Levi Wolf and Serge Rey. As part of the workshop a geopandas data frame was constructed with one of the columns reporting the median listing price of units in each neighborhood in Berlin:


The attribute `median_pri` stores the listing price of airbnb the neighborhoods (in EUROs)

In [ ]:
gdf.head(10)

Create a histogram to see the distribution of the airbnb prices.

In [ ]:
gdf.hist(bins=20)

Next, we will plot the median airbnb price (`median_pri`) in a choropleth map.

In [ ]:
gdf.plot(column='median_pri')

Plot using quantiles classification and green-blue color scheme. [Other color schemes](https://matplotlib.org/tutorials/colors/colormaps.html)

In [ ]:
gdf.plot(column='median_pri', scheme='quantiles', k=5, cmap='GnBu', legend=True)

# gdf.plot(column='median_pri', scheme='equal_interval', k=5, cmap='OrRd', legend=True)

## Spatial Autocorrelation

The choropleth map allows us to visually observe spatial pattern. If the spatial distribution of the prices was random, then we should not see any clustering of similar values on the map. However, our visual system is drawn to the darker clusters in the south west as well as the center, and a concentration of the lighter hues (lower prices) in the north central and south east.

Our brains are very powerful pattern recognition machines. However, sometimes they can be too powerful and lead us to detect false positives, or patterns where there are no statistical patterns. This is a particular concern when dealing with visualization of irregular polygons of differning sizes and shapes.

The concept of spatial autocorrelation relates to the combination of two types of similarity: spatial similarity and attribute similarity. Although there are many different measures of spatial autocorrelation, they all combine these two types of simmilarity into a summary measure.

Let's use PySAL to generate these two types of similarity measures.

# Spatial Similarity

We have already encountered spatial weights in a previous notebook. In spatial autocorrelation analysis, the spatial weights are used to formalize the notion of spatial similarity (whether things are located near each other).

![](images/fig15.jpg)

Get the attribute of median price

In [ ]:
y = gdf['median_pri']

Create a spatial weight matrix based on queen contiguity.

In [ ]:
wq =  lps.weights.Queen.from_dataframe(gdf)
wq.transform = 'r'

Calculate the spatial lag of `median_pri`, which are averages of airbnb prices of the neighbors.

In [ ]:
ylag = lps.weights.lag_spatial(wq, y)

gdf['lag_median_pri'] = ylag

Plot the spatial lags of polygons (neighborhoods) in Berlin.

The following plot shows the spatial lags in each polygon. The dark polygons are places where the average airbnb price in the neighborhood is high.

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
gdf.plot(column='lag_median_pri', scheme='quantiles', k=5, cmap='OrRd', legend=True, ax=ax)
plt.title("Spatial Lag Median Price (Quintiles)")

plt.show()

The quintile map for the spatial lag tends to enhance the impression of value similarity in space. It is, in effect, a local smoother.

The following code plots the prices and the spatial lags of median price.

In [ ]:
f,ax = plt.subplots(1,2,figsize=(2.16*4,4))
gdf.plot(column='median_pri', ax=ax[0], edgecolor='k',scheme="quantiles",  k=5, cmap='GnBu')
ax[0].axis(gdf.total_bounds[np.asarray([0,2,1,3])])
ax[0].set_title("Price")


gdf.plot(column='lag_median_pri', ax=ax[1], edgecolor='k',scheme='quantiles', cmap='GnBu', k=5)
ax[1].axis(gdf.total_bounds[np.asarray([0,2,1,3])])
ax[1].set_title("Spatial Lag Price")

ax[0].axis('off')
ax[1].axis('off')
plt.show()

## Global Spatial Autocorrelation

From the above fiture we can observe that the spatial distributions of the variable (price) and the spatial lag of the variable are similar, both clustered in city center. 

From the observation, you can infer a linear relationship between the variable and the spatial lag. The slope of the linear regression line is global spatial autocorrelation.

![](images/fig16.jpg)

Get the columns of median prices and spatial lags of median prices

In [ ]:
price = gdf['median_pri']
lag_price = gdf['lag_median_pri']

Fit `price` (x: independent variable) and `lag_price` (y: dependent variable) in a linear equation (y=b*x+e)

b is the coefficient (the slope of the regression line) and e is the residual.

In [ ]:
b, e = np.polyfit(price, lag_price, 1)

Plot the scatter plot between the prices and the spatial lags of the prices.

Two lines are added to partition the scatter plot into four quadrats.

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))

plt.plot(price, lag_price, '.', color='firebrick')

 # dashed vert at mean of the price
plt.vlines(price.mean(), lag_price.min(), lag_price.max(), linestyle='--')
 # dashed horizontal at mean of lagged price 
plt.hlines(lag_price.mean(), price.min(), price.max(), linestyle='--')

# red line of best fit using global I as slope
plt.plot(price, e + b*price, 'r')
plt.title('Moran Scatterplot')
plt.ylabel('Spatial Lag of Price')
plt.xlabel('Price')
plt.show()

The spatial autocorrelation is indicated by the slope of the regression line (i.e. b)

The four quadrats in the scatter plot indicates High-High (top-right), Low-Low (bottom-left), High-Low (top-right), and Low-High (bottom-left) polygons.

Instead of calculating the coefficient `b` in a linear regression model, we can use the `moran.Moran` function in the `esda` package to calculate Moran's I

In [ ]:
mi = esda.moran.Moran(gdf['median_pri'], wq)

In [ ]:
mi.I

The p-value of Moran's I. 

Can be interpreted as the probability that the spatial pattern of the prices is random.

The smaller the p-value is, the more significant the test is.

In [ ]:
mi.p_sim

Comparing the Moran's I of the housing price with the distribution of random simulation.

As can be observed in the histogram, the Moran's I of the housing prices is positively deviated from the mean of the random distribution, indicating the spatial autocorrelation is positive (a clustered pattern).

In [ ]:
import seaborn as sbn
sbn.kdeplot(mi.sim, shade=True)
plt.vlines(mi.I, 0, 1, color='r')
plt.vlines(mi.EI, 0,1)
plt.xlabel("Moran's I")

## Local Autocorrelation: Hot Spots, Cold Spots, and Spatial Outliers

The global spatial autocorrelation describes the overal spatial pattern in the entire study area. However, you can observe heterogenous pattersns at different local areas. Some places are clustered, some places are dispersed, some are like random.

Moran's I can be calculated in the focal area centered at each polygon, considering the spatial autocorrelation in the neighborhood of the polygon.

Calculate local Moran's I:

In [ ]:
lmi = esda.moran.Moran_Local(y, wq)

Print which quadrant does each polygon is located. (1:HH, 2:LH, 3:LL, 4:HL)

In [ ]:
li.q

Calculate the p-value of Local Moran's I.

In [ ]:
li.p_sim

Classify the polygons into four types of patterns. 

Only polygons where p < 0.05 are considered significant.

In [ ]:
hotspot = gdf[(li.p_sim<0.05) & (li.q==1)]
coldspot = gdf[(li.p_sim<0.05) & (li.q==3)]
low_high = gdf[(li.p_sim<0.05) & (li.q==2)]
high_low = gdf[(li.p_sim<0.05) & (li.q==4)]

Plot the hot spots.

[Color codes of matplotlib](https://matplotlib.org/2.0.0/examples/color/named_colors.html).

In [ ]:
from matplotlib import colors
f, ax = plt.subplots(1, figsize=(9, 9))

gdf.plot(color='lightgray',ax=ax)
coldspot.plot(color='red',ax=ax)

---

## Question 2

Following the instruction, create a plot to show cold spots (Low-Low), low-high, and high-low polygons in four different maps.

Please assign color in the following way:
 - cold spots: blue
 - low-high: lightblue
 - high-low: pink

## Question 3

Plot the four types of spatial patterns (hot spot, cold spots, low-high, and high-low) in a single map.

Please assign color in the following way:
 - hot spots: red
 - cold spots: blue
 - low-high: lightblue
 - high-low: pink